## Scraping data

Combine results were scraped from <a href = 'https://www.pro-football-reference.com/draft/2018-combine.html'> Pro Football Reference </a>

In [2]:
import re,urllib
from bs4 import BeautifulSoup
from datetime import date
import requests

In [38]:
def getPlayerFromRow(r):
    row = r.find_all('th')[0].text.strip() 
    cols = r.find_all('td')
    for i in range(len(cols)): 
        if i== len(cols)-1:
            if cols[i].text =='':
                row = row + ','
            else:
                row = row + ',' + cols[i].text.split('/')[1][:-3]
            continue
        if i==2:
            continue
        else:
            row = row + ','+cols[i].text
    return row

In [39]:
from selenium import webdriver
print('Opening Web Browser...')
browser = webdriver.Chrome()
YEAR = 2018
header = 'Player,Position,School,Height,Weight,Forty,Vertical,Bench,Broad,Cones,Shuttle,Round,Year'
while YEAR>=2000:
    with open('combine_{}.csv'.format(YEAR),'a') as f:
        f.write(header+'\n')
        print('Loading data from {} Combine'.format(YEAR),end='')
        browser.get('https://www.pro-football-reference.com/draft/{}-combine.htm'.format(YEAR))
        data = BeautifulSoup(browser.execute_script("return document.body.innerHTML"),'lxml')
        combine = data.find(id='combine').find_all('tbody')[0]
        rows = combine.find_all('tr')
        for i in range(len(rows)):
            if(i%(len(rows)//10)==0 and i!=0):
                print('.',end='')
            f.write(getPlayerFromRow(rows[i])+','+str(YEAR)+'\n')
        print('')
    YEAR = YEAR-1
browser.quit()

Opening Web Browser...
Loading data from 2018 Combine..........
Loading data from 2017 Combine..........
Loading data from 2016 Combine..........
Loading data from 2015 Combine..........
Loading data from 2014 Combine..........
Loading data from 2013 Combine..........
Loading data from 2012 Combine.........
Loading data from 2011 Combine..........
Loading data from 2010 Combine..........
Loading data from 2009 Combine..........
Loading data from 2008 Combine..........
Loading data from 2007 Combine..........
Loading data from 2006 Combine..........
Loading data from 2005 Combine..........
Loading data from 2004 Combine..........
Loading data from 2003 Combine..........
Loading data from 2002 Combine..........
Loading data from 2001 Combine..........
Loading data from 2000 Combine..........


In [74]:
import pandas as pd
YEAR = 2018
combine_data = pd.read_csv('combine_{}.csv'.format(YEAR))
while YEAR>=2001:
    YEAR = YEAR - 1
    combine_data = combine_data.append(pd.read_csv('combine_{}.csv'.format(YEAR)))
combine_data = combine_data[combine_data.Player!='Player']
combine_data['Height'] = combine_data['Height'].apply(lambda x: int(x.split('-')[0])*12+int(x.split('-')[1]))
combine_data = combine_data.reset_index(drop = True)

In [77]:
test = combine_data[combine_data.index%5==0]
training = combine_data[combine_data.index%5!=0]

In [78]:
training

,Player,Position,School,Height,Weight,Forty,Vertical,Bench,Broad,Cones,Shuttle,Round,Year
1,Kemoko Turay,EDGE,Rutgers,77,253.0,4.65,NaN,NaN,NaN,NaN,NaN,2.0,2018.0
2,Josh Adams,RB,Notre Dame,74,213.0,NaN,NaN,18.0,NaN,NaN,NaN,NaN,2018.0
3,Ola Adeniyi,EDGE,Toledo,74,248.0,4.83,31.5,26.0,NaN,7.21,4.28,NaN,2018.0
4,Jordan Akins,TE,Central Florida,75,249.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2018.0
6,Austin Allen,QB,Arkansas,72,210.0,4.81,29.5,NaN,112.0,7.18,4.48,NaN,2018.0
7,Brian Allen,C,Michigan St.,73,298.0,5.34,26.5,27.0,99.0,7.81,4.71,4.0,2018.0
8,Josh Allen,QB,Wyoming,77,237.0,4.75,33.5,NaN,119.0,6.90,4.40,1.0,2018.0
9,Marcus Allen,S,Penn St.,74,202.0,NaN,37.0,15.0,127.0,NaN,4.32,NaN,2018.0
11,Troy Apke,S,Penn St.,74,198.0,4.34,41.0,16.0,131.0,6.56,4.03,NaN,2018.0
12,Dorance Armstrong,EDGE,Kansas,76,257.0,4.87,30.0,20.0,118.0,7.12,4.23,NaN,2018.0


In [59]:
wrs = combine_data[combine_data.Position=='WR']
rbs = combine_data[combine_data.Position=='RB']

In [70]:
import numpy as np
np.mean(rbs['Height'].values)

70.553703703703704

In [81]:
positions = training['Position'].unique()

In [82]:
len(positions)

24

In [89]:
mean_heights = [np.mean(training[training.Position==pos]['Height'].values) for pos in positions]
std_heights = [np.std(training[training.Position==pos]['Height'].values) for pos in positions]
mean_weights = [np.mean(training[training.Position==pos]['Weight'].values) for pos in positions]
std_weights = [np.std(training[training.Position==pos]['Weight'].values) for pos in positions]
probs = [len(training[training.Position==pos])/len(training) for pos in positions]

In [104]:
from scipy.stats import norm
likelihoods = [[p*norm.pdf((Hx-h)/sh)*norm.pdf((Wx-w)/sw) for p,h,sh,w,sw in zip(probs,mean_heights,std_heights,mean_weights,std_weights)] for Hx,Wx in test[['Height','Weight']].values]

C:\Users\raghu\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app
C:\Users\raghu\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
C:\Users\raghu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
C:\Users\raghu\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


In [105]:
results = [positions[a.index(max(a))] for a in likelihoods ] 

In [108]:
np.average([1 if r==s else 0 for r,s in zip(results,test['Position'].values)])

0.46543408360128619

In [107]:
test

,Player,Position,School,Height,Weight,Forty,Vertical,Bench,Broad,Cones,Shuttle,Round,Year
0,Dante Pettis,WR,Washington,72,186.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2018.0
5,Jaire Alexander,CB,Louisville,71,192.0,4.38,35.0,14.0,127.0,6.71,3.98,1.0,2018.0
10,Mark Andrews,TE,Oklahoma,77,256.0,4.67,31.0,17.0,113.0,7.34,4.38,3.0,2018.0
15,John Atkins,DT,Georgia,76,321.0,5.38,24.0,NaN,89.0,7.95,4.75,NaN,2018.0
20,Kalen Ballage,RB,Arizona State,73,228.0,4.46,33.5,15.0,122.0,6.91,4.35,4.0,2018.0
25,Nick Bawden,FB,San Diego St.,74,245.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018.0
30,Quin Blanding,S,Virginia,74,215.0,4.63,31.5,NaN,122.0,NaN,NaN,NaN,2018.0
35,Tony Brown,CB,Alabama,72,198.0,4.35,31.5,14.0,126.0,6.78,4.11,NaN,2018.0
40,Jason Cabinda,ILB,Penn St.,73,239.0,NaN,NaN,19.0,NaN,NaN,NaN,NaN,2018.0
45,Alex Cappa,OT,Humboldt St.,78,305.0,5.39,26.5,24.0,97.0,8.04,4.84,NaN,2018.0
